In [ ]:
# library imports
from ROOT import gInterpreter, TChain, TFile, EnableImplicitMT, RDataFrame, GetThreadPoolSize
import numpy as np
import matplotlib.pyplot as plt
import json

# Function module import
import FunctionDefines as rdf_functions
import Utils

# Declaration of C++ code
gInterpreter.Declare(rdf_functions.GetCppCode())

In [ ]:
##% Read processes waveform files as RDF
# Read the v1751 files in the right order
wfs_chain = TChain("ProcWfTree")
wfs_chain.Add("./data_files/processed_wfs_WFirm*.root")

proc_df = RDataFrame(wfs_chain)
transform_df = proc_df

# define the list of chamber channel names
ch_lst = []
for digi_name in ["v1751", "v1752"]:
    [ch_lst.append("{0}_ch0{1}".format(digi_name, i)) for i in range(6)]

In [ ]:
##% Studying the application of a Haar wavelet filter

# Define the original waveform integrals
for ch_name in ch_lst:
    transform_df = transform_df.Define(
        "{0}_SB_int".format(ch_name),
        "Numba::d_GetTotInt({0}_SB,0.02)".format(ch_name),
    )

In [ ]:
for ch_name in ch_lst:
    # extract the right channel integrals each time
    filter_dict = transform_df.Filter(
        "Numba::f_FFTInRange({0}_FFTmag, 3,0.,0.4)".format(ch_name)
    ).AsNumpy(["{0}_SB_int".format(ch_name), "{0}_WF_int".format(ch_name)])
    
    fig, ax = plt.subplots(figsize=(10, 6), facecolor="white")
    plt.title("Collected charge distributions ({0})".format(ch_name), fontsize=18)
    plt.xlabel("Collected charge [nC]", fontsize=16)
    plt.ylabel("Events", fontsize=16)
    plt.yscale("log")
    ax.minorticks_on()
    plt.grid()
    plt.tight_layout()
    plt.hist(
        filter_dict["{0}_SB_int".format(ch_name)],
        bins=500,
        range=[-0.1, 0.2],
        histtype="step",
        linewidth=2,
        label="Initial waveform integrals",
    )
    plt.hist(
        filter_dict["{0}_WF_int".format(ch_name)],
        bins=500,
        range=[-0.1, 0.2],
        histtype="step",
        linewidth=2,
        label="Wavelet-filtered ($[8\cdot 10^{-3},5\cdot 10^{-2}]$) integrals",
    )
    plt.legend(fontsize=16)
    plt.savefig("./plots/WFirm_timing/{0}_WF_integrals.pdf".format(ch_name))

In [ ]:
##% Computing the filtered pedestal HWHM using PyRoot (excluding the 0 bin)
from ROOT import TCanvas, TLine

fit_d = {
    "fit_pars": {"h_range": [-0.1, 0.2], "bins": 500, "f_range": [-0.02, 0.02]},
    "HWHM": {},
}

for ch_name in ch_lst:
    canvas = TCanvas("canvas")
    canvas.cd()
    canvas.SetLogy()

    ch_histo = transform_df.Filter(
        "Numba::f_FFTInRange({0}_FFTmag, 3,0.,0.4)".format(ch_name)
    ).Histo1D(
        (
            "f_int_distr",
            "Filtered histogram HWHM",
            fit_d["fit_pars"]["bins"],
            fit_d["fit_pars"]["h_range"][0],
            fit_d["fit_pars"]["h_range"][1],
        ),
        "{0}_WF_int".format(ch_name),
    )
    start_bin = (
        ch_histo.FindFirstBinAbove(
            ch_histo.GetBinContent(ch_histo.GetMaximumBin() - 1) / 2
        )
        - 1
    )
    end_bin = (
        ch_histo.FindLastBinAbove(
            ch_histo.GetBinContent(ch_histo.GetMaximumBin() - 1) / 2
        )
        + 1
    )
    print(
        ch_histo.GetBinContent(ch_histo.GetMaximumBin() - 1) / 2,
        "-",
        ch_histo.GetBinContent(ch_histo.GetMaximumBin() - 1) / 2,
    )
    hwhm = ch_histo.GetBinCenter(end_bin) - ch_histo.GetBinCenter(start_bin)
    fit_d["HWHM"].update({ch_name: hwhm})
    # plot the histograms and the HWHM range
    ch_histo.GetXaxis().SetTitle("Filtered wf. integral [nC]")
    ch_histo.GetYaxis().SetTitle("Events")
    start_line = TLine(
        ch_histo.GetBinCenter(start_bin),
        0.0,
        ch_histo.GetBinCenter(start_bin),
        ch_histo.GetBinContent(start_bin),
    )
    start_line.SetLineColor(2)
    end_line = TLine(
        ch_histo.GetBinCenter(end_bin),
        0.0,
        ch_histo.GetBinCenter(end_bin),
        ch_histo.GetBinContent(end_bin),
    )
    end_line.SetLineColor(2)

    ch_histo.Draw()
    start_line.Draw("same")
    end_line.Draw("same")
    canvas.Draw()
    canvas.SaveAs("./plots/WFirm_timing/{0}_HWFM.pdf".format(ch_name))

# Convert and write dictionary to JSON file
with open("./plots/WFirm_timing/HWHM_pars_WFirm.json", "w") as outfile:
    json.dump(fit_d, outfile)

In [ ]:
##% Check the distribution of tracker hits

# load the HWFM dictionary
with open("./plots/pedestal_analysis/HWHM_pars.json", "r") as infile:
    hwhm_dict = json.load(infile)
# set some constants
ch_name = "v1752_ch00"
# (z0-top_tr)+(b_c-t_tr)+h_scr+h_c-pl_off
ch_height = (-134 + 107.3) + 20.3 + 0.2 + 5.6 - (1.5)

XY_df = (
    transform_df.Filter(
        "Numba::f_ValAboveThr({0}_WF_int,{1}) && Numba::f_FFTInRange({0}_FFTmag, 3,0.,0.4) && tr_pars_r0_x>-999 && Numba::f_DeltaAboveVal(v1752_ch07_SB,0.05)".format(
            ch_name, 3 * hwhm_dict["HWHM"][ch_name]
        )
    )
    .Define(
        "X_proj",
        "Numba::d_GetXYProjection(tr_pars_r0_x,tr_pars_sx,{0})".format(ch_height),
    )
    .Define(
        "Y_proj",
        "Numba::d_GetXYProjection(tr_pars_r0_y,tr_pars_sy,{0})".format(ch_height),
    )
    .Define(
        "Delta_t",
        "Numba::d_GetChamberT10p({0}_SB)-Numba::d_GetPMTT10p(v1752_ch07_SB)".format(
            ch_name
        ),
    )
)

proj_dict = XY_df.AsNumpy(["X_proj", "Y_proj", "Delta_t"])
np_XY_proj = np.column_stack((proj_dict["X_proj"], proj_dict["Y_proj"]))

fig, ax = plt.subplots(figsize=(10, 8), facecolor="white")
plt.axis("equal")
plt.title("$\Delta T$ for (1752_ch00), Int${}_{wf}>3\cdot$HWHM", fontsize=18)
plt.xlabel("$X_{reco}$ [cm]", fontsize=18)
plt.ylabel("$Y_{reco}$ [cm]", fontsize=18)
plt.tight_layout()
plt.scatter(
    np_XY_proj[:, 0],
    np_XY_proj[:, 1],
    c=proj_dict["Delta_t"],
    vmin=0.0,
    vmax=200,
    cmap="plasma",
    s=3,
    label="ch_00 cluster",
)
cbar = plt.colorbar()
cbar.set_label(label="$\Delta T$ [ns]",fontsize=12)
plt.savefig("./plots/pedestal_analysis/XYDT_plt.pdf")

In [ ]:
##% Apply PCA to get rough estimates of the wire parameters

# load the HWFM dictionary
with open("./plots/pedestal_analysis/HWHM_pars.json", "r") as infile:
    hwhm_dict = json.load(infile)

ch_name = "v1752_ch00"
# (z0-top_tr)+(b_c-t_tr)+h_scr+h_c-pl_off
ch_height = (-134 + 107.3) + 20.3 + 0.2 + 1.3 + 0.5 + (2 * 1)

XY_df = (
    transform_df.Filter(
        "Numba::f_ValAboveThr({0}_WF_int,{1}) && Numba::f_FFTInRange({0}_FFTmag, 3,0.,0.4) && tr_pars_r0_x>-999 && Numba::f_DeltaAboveVal(v1752_ch07_SB,0.05)".format(
            ch_name, 3 * hwhm_dict["HWHM"][ch_name]
        )
    )
    .Define(
        "X_proj",
        "Numba::d_GetXYProjection(tr_pars_r0_x,tr_pars_sx,{0})".format(ch_height),
    )
    .Define(
        "Y_proj",
        "Numba::d_GetXYProjection(tr_pars_r0_y,tr_pars_sy,{0})".format(ch_height),
    )
)

pca_mean, pca_components = Utils.GetPCAParameters(XY_df)

In [ ]:
ch_height0 = (-134 + 107.3) + 27.3  # + 20.3 + 0.2 + 1.3 +0.5+(2*1)+4
r_wire = np.array([18.42753051011351, 19.50202293171639,ch_height])
e_wire = np.array([np.cos(-7.433052706454731*(np.pi/180)),np.sin(-7.433052706454731*(np.pi/180))])
print(r_wire)
print(e_wire)

In [ ]:
##% First attempt at dist-T plot

# # load the HWFM dictionary
# with open("./plots/pedestal_analysis/HWHM_pars.json", "r") as infile:
#     hwhm_dict = json.load(infile)
# set some constants
ch_name = "v1752_ch00"

# ch_height = (-134 + 107.3) + 20.3 + 0.2 + 1.3 +0.5+(2*1)+3

# r_wire = np.append(pca_mean, ch_height)
# e_wire = pca_components[0]
# print(r_wire)
# print(e_wire)

XY_df = (
    transform_df.Filter(
        "Numba::f_ValAboveThr({0}_WF_int,{1}) && Numba::f_FFTInRange({0}_FFTmag, 3,0.,0.4) && tr_pars_r0_x>-999 && Numba::f_DeltaAboveVal(v1752_ch07_SB,0.05)".format(
            ch_name, 3 * hwhm_dict["HWHM"][ch_name]
        )
    )
    .Define(
        "Delta_t_SB",
        "Numba::d_GetChamberT10p({0}_SB)-Numba::d_GetPMTT10p(v1752_ch07_SB)".format(
            ch_name
        ),
    )
    .Define(
        "Delta_t_WF",
        "Numba::d_GetChamberT10p({0}_WFirm)-Numba::d_GetPMTT10p(v1752_ch07_SB)".format(
            ch_name
        ),
    )
    .Define(
        "track_dist",
        "Numba::d_TrackWireDist(tr_pars_r0_x,tr_pars_r0_y,tr_pars_sx,tr_pars_sy,{0},{1},{2},{3},{4})".format(
            r_wire[0], r_wire[1], r_wire[2], e_wire[0], e_wire[1]
        ),
    )
    
)

td_dict = XY_df.AsNumpy(["Delta_t_SB", "Delta_t_WF","track_dist"])

fig, ax = plt.subplots(figsize=(10, 8), facecolor="white")
plt.hist2d(td_dict["track_dist"], td_dict["Delta_t_SB"],bins=[100,100],range=[[0.0, 2.5],[0, 200]])
# plt.ylim(0, 200)
# plt.xlim(0.0, 2.5)
plt.title("T-distance relation", fontsize=18)
plt.xlabel("$D_{track}$ [cm]", fontsize=18)
plt.ylabel("$\Delta T$ [ns]", fontsize=18)
ax.minorticks_on()
plt.colorbar()
plt.tight_layout()
plt.savefig("./plots/WFirm_timing/TD_rel_hist_SB_Fits.pdf")

fig, ax = plt.subplots(figsize=(10, 8), facecolor="white")
plt.hist2d(td_dict["track_dist"], td_dict["Delta_t_WF"],bins=[100,100],range=[[0.0, 2.5],[0, 200]])
# plt.ylim(0, 200)
# plt.xlim(0.0, 2.5)
plt.title("T-distance relation with firm thresholding", fontsize=18)
plt.xlabel("$D_{track}$ [cm]", fontsize=18)
plt.ylabel("$\Delta T$ [ns]", fontsize=18)
ax.minorticks_on()
plt.colorbar()
plt.tight_layout()
plt.savefig("./plots/WFirm_timing/TD_rel_hist_WT_Fits.pdf")

In [ ]:
##% Check the residues between SB and Filtered t_10%
fig, ax = plt.subplots(figsize=(10, 8), facecolor="white")
plt.hist((td_dict["Delta_t_WF"]-td_dict["Delta_t_SB"]),bins=500,range=[-250,250])

In [ ]:
##% Checks with T_10%

# load the HWFM dictionary
with open("./plots/pedestal_analysis/HWHM_pars.json", "r") as infile:
    hwhm_dict = json.load(infile)

ch_name = "v1752_ch00"
# Take the usual cut on the events
Time_df = (
    transform_df.Filter(
        "Numba::f_ValAboveThr({0}_WF_int,{1}) && Numba::f_FFTInRange({0}_FFTmag, 3,0.,0.4) && tr_pars_r0_x>-999 && Numba::f_DeltaAboveVal(v1752_ch07_SB,0.05)".format(
            ch_name, 3 * hwhm_dict["HWHM"][ch_name]
        )
    )
    .Define(
        "T_chSB",
        "Numba::d_GetChamberT10p({0}_SB)".format(ch_name),
    ).Define(
        "T_chWF",
        "Numba::d_GetChamberT10p({0}_WF)".format(ch_name),
    )
    .Define(
        "T_PMT",
        "Numba::d_GetPMTT10p(v1752_ch07_SB)".format(ch_name),
    )
)

t_dict = Time_df.AsNumpy(["{0}_SB".format(ch_name),"{0}_WF".format(ch_name), "T_chSB","T_chWF"])

t_srsSB = np.arange(np.asarray(t_dict["{0}_SB".format(ch_name)][0]).shape[0]) * 1e-9
t_srsWF = np.arange(np.asarray(t_dict["{0}_WF".format(ch_name)][0]).shape[0]) * 1e-9
for idx in range(70,80):
    fig, ax = plt.subplots(figsize=(12, 8), facecolor="white")
    plt.title("WF-Waveforms and $t_{10\%}$ (v1752_ch00)", fontsize=18)
    plt.xlabel("Time [ns]", fontsize=16)
    plt.ylabel("Amp [V]", fontsize=16)
    ax.minorticks_on()
    plt.grid()
    plt.tight_layout()
    plt.plot(t_srsSB,t_dict["{0}_SB".format(ch_name)][idx],label="SB-Waveform")
    plt.plot(t_srsWF,t_dict["{0}_WF".format(ch_name)][idx],label="WF-Waveform")
    plt.plot([t_dict["T_chSB"][idx]* 1e-9,t_dict["T_chSB"][idx]* 1e-9],[-0.01,0.05],label="SB $t_{10\%}$",linewidth=3)
    plt.plot([t_dict["T_chWF"][idx]* 1e-9,t_dict["T_chWF"][idx]* 1e-9],[-0.01,0.05],label="WF $t_{10\%}$",linewidth=2)
    plt.legend(fontsize=16)
    #plt.savefig("./plots/other_checks/evt_{0}_WF_t10p.png".format(idx))

In [ ]:
from ROOT import TCanvas

hist = (
    XY_df
    .Histo1D(
        (
            "Selection_checks",
            "Track - wire distance ({0})".format(ch_name),
            100,
            0.0,
            10,
        ),
        "track_dist",
    )
)
hist.GetXaxis().SetTitle("Track - wire distance [cm]")
hist.GetYaxis().SetTitle("Events")
canvas = TCanvas("canvas")
canvas.cd()
hist.Draw()
canvas.Draw()
canvas.SaveAs("./plots/pedestal_analysis/Distance_distribution.pdf")

In [ ]:
##% Checks with the Polar angle
from ROOT import TCanvas

ch_name = "v1752_ch00"

ch_height = (-134 + 107.3) + 20.3 + 0.2 + 1.3 + 0.5 + (2 * 1)

r_wire = np.append(pca_mean, ch_height)
e_wire = pca_components[0]
# print(r_wire[0], ",", r_wire[1], ", ", r_wire[2], ", ", e_wire[0], ", ", e_wire[1])

hist_polar = (
    transform_df.Filter(
        "Numba::f_ValAboveThr({0}_WF_int,{1}) && Numba::f_FFTInRange({0}_FFTmag, 3,0.,0.4) && tr_pars_r0_x>-999 && Numba::f_DeltaAboveVal(v1752_ch07_SB,0.05)".format(
            ch_name, 3 * hwhm_dict["HWHM"][ch_name]
        )
    )
    .Define(
        "PolAngle", "Numba::d_PolarAngle(tr_pars_sx,tr_pars_sy)*(180/{0})".format(np.pi)
    )
    .Define(
        "Delta_t_SB",
        "Numba::d_GetChamberT10p({0}_SB)-Numba::d_GetPMTT10p(v1752_ch07_SB)".format(
            ch_name
        ),
    )
).Histo2D(
    (
        "#DeltaT vs. #theta",
        "#DeltaT vs. #theta ({0})".format(ch_name),
        50,
        0.0,
        12,
        50,
        0.0,
        200,
    ),
    "PolAngle",
    "Delta_t_SB",
)

hist_polar.GetXaxis().SetTitle("Polar angle [deg]")
hist_polar.GetYaxis().SetTitle("#DeltaT [ns]")
canvas = TCanvas("canvas")
canvas.cd()
hist_polar.Draw("colz")
canvas.Draw()
#canvas.SaveAs("./plots/pedestal_analysis/PolarAngle_distr.pdf")